In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 03
### Team Member Names: Sherry, Kelly, Ifan
### Team Strategy Chosen: SAFE

In [4]:
max_stocks = 25 #The total number of stocks we can possibly have
tickers_df = pd.read_csv('Tickers_Example.csv')
#When the CSV file is read, it takes the first ticker and makes it the name of the column, so we need to extract it
tickers_list = list(tickers_df.columns)
#Now add the rest of the tickers in after it:
tickers_list += tickers_df.iloc[:, 0].tolist()
tickers_num = len(tickers_list)

#Dates to pull closing data from, change as needed
start_date = '2020-01-01'
end_date = '2022-01-02'

# Dictionary of all the stocks
stocks_dict = {}
stocks_full_info = {}

# Function produces the daily closing prices of a stock
def get_closing_prices(ticker, start, end):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end) #Daily
    return hist.Close

for i in range (tickers_num):
    stocks_dict[tickers_list[i]] = get_closing_prices(tickers_list[i], start_date, end_date)
    stocks_full_info[tickers_list[i]] = yf.Ticker(tickers_list[i])

# Create DataFrame
all_prices = pd.DataFrame(stocks_dict)

tickers_list_temp = tickers_list.copy()
#Some tickers are delisted, so we drop those columns entirely
for i in range (tickers_num):
    if all_prices[tickers_list[i]].isnull().all():
        all_prices.drop(columns=[tickers_list[i]], inplace=True)
        tickers_list_temp.remove(tickers_list[i]) #Getting rid of the tickers from our list too to match
        stocks_full_info.pop(tickers_list[i]) #Do the same to the dictionary of yfinance ticker objects

tickers_list = tickers_list_temp.copy()
tickers_num = len(tickers_list)

#Drop the rows at the bottom of the DataFrame where only a few stocks have values and where the dates are strange
all_prices = all_prices.dropna() 

#print(tickers_list)

#Display the entire DataFrame, so that we can manually check to make sure the values are sensible
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(all_prices)

FileNotFoundError: [Errno 2] No such file or directory: 'Tickers_Example.csv'

In [3]:
# Remove invalid stocks per assignment requirement (ones outside of the U.S & ones with average monthly volume of less than 200,000 shares)

index_difference = []

#Function runs for approx. 9 min for yfinance API
def filter_market(df):
    df_transposed = df.T
    mkt_lst = []
    
    for i in range(len(df_transposed)):
        # Populate market list
        mkt_lst.append(stocks_full_info[tickers_list[i]].info['market'])
    
    df_transposed['Market'] = mkt_lst
    
    # Drops the stocks in markets that are not in the U.S
    df_transposed.drop(df_transposed[df_transposed.Market != "us_market"].index, inplace=True)
    df_transposed = df_transposed.drop('Market', axis=1)
    return_df = df_transposed.T

    # Tracks the indices of the values that were dropped
    index_difference.append(df.difference(return_df.T, sort=False))

    return return_df

# Drops the stocks with volumes less than average monthly volume of 200,000
def filter_volume(df):

    start = "2022-01-01"
    end = "2022-10-31"

    df_transposed = df.T
    volume_lst = []

    for i in range(len(df_transposed)):
        ticker_hist = stocks_full_info[tickers_list[i]].history(start=start, end=end)
        volume_hist = ticker_hist.loc[(ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        volume_lst.append(volume_hist['Volume'].mean()*30)

    # Drops the stocks with volumes less than average monthly volume of 200,000
    df_transposed["Volume"] = volume_lst

    df_transposed.drop(df_transposed[df_transposed.Volume < 200000].index, inplace=True)
    df_transposed = df_transposed.drop('Volume', axis=1)
    return_df = df_transposed.T

    # Tracks the indices of the values that were dropped
    index_difference.append(df.difference(return_df.T, sort=False))

    return return_df

all_prices = all_prices.loc[filter_market(all_prices)]
all_prices = all_prices.loc[filter_volume(all_prices)]

# Drops stocks with more than specified number of NaN values
max_number_of_nans = 20
all_prices = all_prices.loc[:, (all_prices.isnull().sum(axis=0) <= max_number_of_nans)]

KeyError: 0

In [ ]:
# Reset previously defined variables
all_prices = filter_volume(all_prices)

for col in all_prices.columns:
     tickers_list[i] = col

for i in range(len(tickers_list)):
    stocks_full_info = stocks_full_info.pop(tickers_list[index_difference[i]])

tickers_num = len(tickers_list)

In [38]:
#Function to get the beta of a stock against a given market
def stock_beta(stock_ticker, market_index):
    
    #Get basic info on both the stock and the market
    index_ticker = yf.Ticker(market_index)
    stock_ticker_object = yf.Ticker(stock_ticker)
    stock_hist = stock_ticker_object.history(start=start_date, end=end_date)
    MarketIndex_hist = index_ticker.history(start=start_date, end=end_date)
    
    #Get the closing prices of both the stock and market
    prices = pd.DataFrame(stock_hist['Close'])
    prices.columns = [stock_ticker]
    prices[market_index] = MarketIndex_hist['Close']
    
    #Daily returns of both stock and market
    daily_returns = prices.pct_change()
    daily_returns.drop(index=daily_returns.index[0], inplace=True) #Drop the first entry since it's NaN
    
    #Calculate market variance
    MarketVar = daily_returns[market_index].var()
    
    #Now calculate the beta by taking the covariance
    beta = daily_returns.cov() / MarketVar
    
    return beta.iat[0,1] #To get the value we need, don't care about the others

In [47]:
daily_returns = all_prices.pct_change()

#New DataFrame to hold our std and beta stats
stats_df = pd.DataFrame(daily_returns.std())
stats_df.columns = ['std']

# Calculate BETA against S&P500
MarketIndex = '^GSPC' #This is the symbol yfinance uses for the S&P 500
beta_list = []

for i in range (tickers_num):
    beta_list.append(stock_beta(tickers_list[i], MarketIndex))

stats_df['beta'] = beta_list
stats_df

,std,beta
AAPL,0.014136,1.146468
ABBV,0.012481,0.696572
ABT,0.012989,0.812703
ACN,0.012157,1.050213
AIG,0.017866,1.466294
AMZN,0.014939,0.726506
AXP,0.017102,1.428167
BA,0.020791,1.683175
BAC,0.015323,1.335032
BIIB,0.033740,0.807932


In [63]:
#Get a pandas series of the beta and sd, sorted in ascending order, then put that in a DataFrame
beta_df = pd.DataFrame(stats_df['beta'].sort_values(ascending=True))
std_df = pd.DataFrame(stats_df['std'].sort_values(ascending=True))

#Now we give each stock points in each category using a list
std_points = []
beta_points = []
for i in range (tickers_num):
    std_points.append(tickers_num - i) #So the stocks with the lowest std get the highest number of points
    beta_points.append(tickers_num - i) #So the stocks with the lowest beta get the highest number of points

#Add our lists to our DataFrames as new columns
beta_df['points'] = beta_points
std_df['points'] = std_points

#Store all the necessary points values we need, adding the points from beta and std together
total_points = pd.DataFrame(std_df['points'])
total_points.columns = ['std']
total_points['beta'] = pd.DataFrame(beta_df['points'])
total_points['total'] = total_points['std'] + total_points['beta']

#Now we isolate the total points and sort that series from highest to lowest before turning it back into a DataFrame
#Since we want to prioritize the stocks with the highest number of total points
final_points = pd.DataFrame(total_points['total'].sort_values(ascending=False))
final_points.reset_index(inplace=True)
final_points.columns = ['tickers', 'points']
final_points

,tickers,points
0,MON,114
1,PG,105
2,BMY,104
3,CL,103
4,KO,99
5,COST,98
6,RY.TO,94
7,ABBV,91
8,PEP,90
9,TD.TO,85


In [70]:
selected_tickers = []
sector_count = {}
sector_max = 5

for i in range (tickers_num):
    #We only want choose the maximum number of stocks we can (initiated in the very beginning of the code), not the
    # total number of stocks we have, so if we have enough stocks, do nothing
    if len(selected_tickers) < max_stocks: 
        sector = stocks_full_info[tickers_list[i]].info['sector']
        if sector in sector_count: #If there is a key in sector_count that matches the sector of our stock
            #If our current number of stocks is less than our maximum, then we add it, otherwise we just skip over it
            # and do nothing
            if sector_count[sector] < sector_max: 
                selected_tickers.append(final_points.tickers[i])
                sector_count[sector] += 1
        else: #So if there is no sector in our dictionary that matches that of the stock
            selected_tickers.append(final_points.tickers[i])
            sector_count[sector] = 1

selected_tickers
#～ 10 min to load

['MON',
 'PG',
 'BMY',
 'CL',
 'KO',
 'COST',
 'RY.TO',
 'ABBV',
 'PEP',
 'TD.TO',
 'PM',
 'T',
 'SO',
 'MO',
 'CVS',
 'TGT',
 'ABT',
 'UNP',
 'MRK',
 'AMZN',
 'CSCO',
 'PFE',
 'UNH',
 'GOOG',
 'LLY',
 'UPS']

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.